## Import Data

In [1]:
#list the data
!ls Data

2000_housevote_meta.csv   2008_senatevote_meta.csv
2000_housevote_pos.csv	  2008_senatevote_pos.csv
2000_senatevote_meta.csv  2010_housevote_meta.csv
2000_senatevote_pos.csv   2010_housevote_pos.csv
2001_housevote_meta.csv   2010_senatevote_meta.csv
2001_housevote_pos.csv	  2010_senatevote_pos.csv
2001_senatevote_meta.csv  2011_housevote_meta.csv
2001_senatevote_pos.csv   2011_housevote_pos.csv
2002_housevote_meta.csv   2011_senatevote_meta.csv
2002_housevote_pos.csv	  2011_senatevote_pos.csv
2002_senatevote_meta.csv  2012_housevote_meta.csv
2002_senatevote_pos.csv   2012_housevote_pos.csv
2003_housevote_meta.csv   2012_senatevote_meta.csv
2003_housevote_pos.csv	  2012_senatevote_pos.csv
2003_senatevote_meta.csv  2013_housevote_meta.csv
2003_senatevote_pos.csv   2013_housevote_pos.csv
2004_housevote_meta.csv   2013_senatevote_meta.csv
2004_housevote_pos.csv	  2013_senatevote_pos.csv
2004_senatevote_meta.csv  2014_senatevote_meta.csv
2004_senatevote_pos.csv   2014_senatevote_pos.csv
200

In [2]:
!ls

Capstone Project Proposal - 2.pdf  Data Wrangling.ipynb    README.md
Data				   PROPUBLICA_API_KEY.TXT


### Get Pro Publica API working

In [3]:
#Load ProPublica API key without exposing it to github. Request one yourself. They're free.
APIfile = open('./PROPUBLICA_API_KEY.TXT',)
APIkey = APIfile.read().replace('\n', '')
print(type(APIkey))
#print(APIkey)

<class 'str'>


In [4]:
#API request function
import requests

#create API key
headers = {'X-API-Key': APIkey}

#load status codes from API documentation(https://projects.propublica.org/api-docs/congress-api/#errors) into dictionary
status_codes = {
    200:'Successful Connection!',
    400:'Bad Request – Your request is improperly formed',
    403:'Forbidden – Your request did not include an authorization header',
    404:'Not Found – The specified record(s) could not be found',
    406:'Not Acceptable – You requested a format that isn’t json or xml',
    500:'Internal Server Error – We had a problem with our server. Try again later',
    503:'Service Unavailable – The service is currently not working. Please try again later'
    }

def API_Request(end_point, verbose=True):
    
    #send a get request to restful API, print status and return response as a dictionary
    response = requests.get(url+end_point, headers=headers)
    
    #handle exceptions. 
    try:
        #try to get the status code of the respones and set status variable to what it means according status_code dictionary
        status = status_codes[response.status_code]
    except: 
        #if we can't get the status code or we don't recognize it, print an error and return the status.
        status = ('API Failure, unknown status code: '+str(response.status_code))
        return status
    
    #if the function call didn't say to be quiet print the status.
    if verbose == True:
        print(status)
    
    #return json response interpreted as a dictionary. 
    return response.json()

#load base url and test endpoint
url = 'https://api.propublica.org/congress/v1'
test_endpoint = '/members/new.json'

#test that the API is working. 
new_members = API_Request(test_endpoint)
print(new_members.keys()) # When the API works this should give the list of attributes per member
print('status:', new_members['status'])
print('no. results:', new_members['results'][0]['num_results'])

Successful Connection!
dict_keys(['status', 'copyright', 'results'])
status: OK
no. results: 20


### The API is working.  Let's import voting data

In [5]:
import pandas as pd
import numpy as np

def API_attempts(endpoint, attempts, verbose=False):
    '''takes and endpoint and requests the API with it for a given number of times'''  
    
    #loop throught the number of attempts
    for attempt in range(attempts):
    
        #if we are already trying again we'll do it loud
        if attempt > 0: 
            verbose = True
            print('Retry:', attempt)
        
        #send endpoint to API function for request
        response = API_Request(endpoint, verbose)
    
        #if we can't extract the status we'll keep trying, and if it's OK we'll stop
        status = 'failed'
        #extract the status of the API request
        try:
            status = response['status']
        except:
            pass
        if status == 'OK':
            break
        else: # if the status is not OK, say so
            print('Status:', status)

    return response, status

#let's create a function that will import vote positions for a given rollcall vote.
def import_vote(congress, chamber, session, roll_call, verbose=False):
    '''imports vote details and member positions of a given roll_call vote
        takes congress number 102-116 for House, 80-116 for Senate
        chamber(house|senate) session 1 for odd 2 for even-numbered years
        returns a dicitonary of vote positions and API status'''
    #construct endpoint for API request
    call_endpoint = '/'+str(congress)+'/'+chamber.lower()+'/sessions/'+str(session)+'/votes/'+str(roll_call)+'.json'
    
    #let's attempt a few times to account for internet burps
    call_response, status = API_attempts(call_endpoint, 3)  
        
    #tr to extract and return the vote positions
    try:
        vote = call_response['results']['votes']['vote']
        return vote, status
    except:
        raise
        return {}, status 
    

def positions_df(metalist):
    '''return a dataframe of postions for a given list of vote metadata'''
    
    #initialize dataframe
    dfp = pd.DataFrame()
    
    #for every position in the list
    for congress, chamber, session, roll_call in metalist:
        
        #request the roll_call vote from the API
        vote, status = import_vote(congress, chamber, session, roll_call)
        
        #extract the positions from the rollcall vote data
        positions = vote.pop('positions')
        
        #turn the postions into a dataframe
        call_positions = pd.DataFrame(positions)
        
        #if both dataframes we are going to merge are not empty
        if not dfp.empty and not call_positions.empty:
            
            #handle duplicated positions for malformed returns
            dups = call_positions.duplicated()
            dups = dups.sum()
            if dups > 0: 
                call_postions = call_postions.drop_duplicates()
                print('dropped'+dups+'duplicates')
        
            #MERGE THE POSITIONS FOR A GIVEN ROLL CALL VOTE INTO DATAFRAME <- core of function 
            dfp = pd.merge(dfp, call_positions, on=mergelst, how='outer')
            
            #let us know it's working
            print('.', end='')
            
        #If dfp is empty this is the first run and we need to initialize it
        elif dfp.empty: 
            #make it equal to call_positions
            dfp = call_positions
            
            #make a list of the columns that will be used to merge the call positions
            mergelst = dfp.columns.tolist()[:-1]
            
            #move party and state to front of merglist
            for x in ['state','party']:
                mergelst.insert(0, mergelst.pop(mergelst.index(x)))
            
            #alert that we've started
            print('\ndf created on', congress, chamber, session, roll_call, end=' ')
        
        #handle empty postitions dataframe:
        elif call_positions.empty:
            
            #let us know there was a problem
            print('\nno vote postions available for', congress, chamber, session, roll_call, end=' ')
            print('Status:', status, end=' ')
            
            #append a series of Nan values
            ht = dfp.shape[0]
            wd = dfp.shape[1]
            nans = pd.Series([np.nan for _ in range(ht)])
            print ('dfp.shape:', dfp.shape)
            print ('len(nans):', len(nans))
            dfp = pd.concat([dfp, nans], axis = 1) #(nans, ignore_index = True, axis=1)
            print(dfp.iloc[:3,-4:])
        
        #handle unknown issue
        else:
            print('\nunknown error for', congress, chamber, session, roll_call, end=' ')
            print('Status:', status, end=' ')
    
    #create a multindex of partymembers
    dfp = dfp.set_index(mergelst).transpose()
    
    #after transposing use the metalist as the index
    dfp.index = metalist
    
    #return the complete data frame
    return dfp 

def import_month(year, chamber, month):
    '''import vote metadata for a given year or month and return a dataframe '''
    
    #construct endpoint
    call_endpoint = '/'+chamber.lower()+'/votes/'+str(year)+'/'+str(month)+'.json'
    
    #make 3 attempts at calling the API with the endpoint
    response, status = API_attempts(call_endpoint, 3)
    
    #Extract the votes from the response
    votes = response['results']['votes']

    #create a dataframe out of the votes
    votesdf = pd.DataFrame(votes)
    
    if not votesdf.empty:
        votesdf = votesdf.set_index(['congress','chamber','session','roll_call']).sort_index()
    
    #return the months dataframe
    return votesdf

def import_year(year, chamber):
    '''import votes based on dates, return a dataframe of appended votes'''
    
    
    #So we know we started:
    print('importing', chamber, year, end=' ')
    #loop through the months 
    for month in range(1,13):
        
        #import the first month
        if month == 1:
            yeardf = import_month(year, chamber, month)
       
        #import the month of votes as a df and append it to the list 
        else:
            df = import_month(year, chamber, month)
            if not df.empty:
                yeardf = yeardf.append(df)
            else:
                print('skip', end=' ')
        #print something so we know it's working
        print(month, end=' ')
        
    #return the dataframe
    return yeardf

def save_years(start_y, end_y):
    '''take a range of years and save a series of dataframes for those years in both the hosue and senate'''
    #iterate through the range of years
    for year in range(start_y, end_y+1):
        #alternate between senate and house
        for chamber in ['senate','house']:
            #import vote metadata dataframe
            votemetadf = import_year(year, chamber)
            
            #import vote positions for given dataframe
            voteposdf = positions_df(votemetadf.index)
            
            #construct paths
            pathhead = 'Data/'+str(year)+'_'+chamber+'vote_'
            metapath = pathhead + 'meta.csv'
            pospath = pathhead + 'pos.csv'
            
            #announce what we're doing
            print('\nsaving '+str(year)+' '+chamber+' as '+metapath+' and '+pospath)
        
            #save data frames as csv in data/csv directory
            votemetadf.to_csv(metapath)
            voteposdf.to_csv(pospath)
            
            #save data frames as pickle in data/pickle directory?
            
            
#test (remove when working)
print('testing import:')

#senate_month_test = import_month(2019, 'senate', 1)
#senate_month_test

#senate_2019_test = import_year(2019, 'senate')
#senate_2019_test

#positions = positions_df(senate_2019_test.index)
#positions

#save_years(2019, 2019)

#maybe actually write some unit tests?

testing import:


In [6]:
house_2014_test = import_year(2014, 'house')
test_positions = positions_df(house_2014_test.index)
test_positions

importing house 2014 1 2 3 4 5 6 7 8 9 skip 10 11 12 
df created on 113 House 2 1 
no vote postions available for 113 House 2 2 Status: OK dfp.shape: (432, 7)
len(nans): 432
  party state vote_position   0
0     R    AL       Present NaN
1     R    MI       Present NaN
2     R    NV    Not Voting NaN
.........
no vote postions available for 113 House 2 12 Status: OK dfp.shape: (433, 17)
len(nans): 433
  vote_position_y vote_position_x vote_position_y   0
0              No      Not Voting             Yes NaN
1              No             Yes             Yes NaN
2              No             Yes             Yes NaN

no vote postions available for 113 House 2 13 Status: OK dfp.shape: (433, 18)
len(nans): 433
  vote_position_x vote_position_y   0   0
0      Not Voting             Yes NaN NaN
1             Yes             Yes NaN NaN
2             Yes             Yes NaN NaN

no vote postions available for 113 House 2 14 Status: OK dfp.shape: (433, 19)
len(nans): 433
  vote_position_y   0  

party                                               R               \
state                                              AL           MI   
district                                            4            3   
dw_nominate                                     0.361        0.649   
member_id                                     A000055      A000367   
name                               Robert B. Aderholt Justin Amash   
congress chamber session roll_call                                   
113      House   2       1                    Present      Present   
                         2                        NaN          NaN   
                         3                        Yes           No   
                         4                        Yes          Yes   
                         5                        Yes          Yes   
                         6                        Yes          Yes   
                         7                         No           No   
                         8                         No           No   
                         9                         No           No   
                         10                Not Voting          Yes   
                         11                       Yes          Yes   
                         12                       NaN          NaN   
                         13                       NaN          NaN   
                         14                       NaN          NaN   
                         15                       NaN          NaN   
                         16                       NaN          NaN   
                         17                       NaN          NaN   
                         18                       NaN          NaN   
                         19                       NaN          NaN   
                         20                       Yes          Yes   
                         21                       Yes           No   
                         22                        No           No   
                         23                       Yes          Yes   
                         24                       Yes          Yes   
                         25                       Yes           No   
                         26                       Yes          Yes   
                         27                       Yes          Yes   
                         28                       Yes           No   
                         29                        No           No   
                         30                       Yes          Yes   
...                                               ...          ...   
                         535               Not Voting          Yes   
                         536               Not Voting          Yes   
                         537               Not Voting          Yes   
                         538               Not Voting          Yes   
                         539               Not Voting           No   
                         540               Not Voting          Yes   
                         541               Not Voting          Yes   
                         542               Not Voting           No   
                         543               Not Voting           No   
                         544               Not Voting           No   
                         545               Not Voting           No   
                         546               Not Voting          Yes   
                         547               Not Voting          Yes   
                         548               Not Voting           No   
                         549               Not Voting           No   
                         550               Not Voting          Yes   
                         551               Not Voting           No   
                         552                       No           No   
                         553                      Yes          Yes   
          

In [33]:
save_years(2000, 2018)

importing senate 2000 1 2 3 4 5 6 7 skip 8 9 10 11 12 df created .........................................................................................................................................................................................................................................................................................................
saving 2000 senate as Data/2000_senatevote_meta.csv and Data/2000_senatevote_pos.csv
importing house 2000 1 2 3 4 5 6 7 skip 8 9 10 11 12 df created ........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

ValueError: not enough values to unpack (expected 2, got 0)

In [8]:
save_years(2010,2018)

importing senate 2010 1 2 3 4 5 6 7 8 9 skip 10 11 12 df created ..........................................................................................................................................................................................................................................................................................................
saving 2010 senate as Data/2010_senatevote_meta.csv and Data/2010_senatevote_pos.csv
importing house 2010 1 2 3 4 5 6 7 8 9 skip 10 11 12 df created .......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
save_years(2014,2017)

In [32]:
test_df = pd.read_csv('Data/2019_senatevote_pos.csv', header = [0,1,2,3,4], index_col = [0,1,2,3])
test_df

party                                            R             D  \
state                                           TN            WI   
dw_nominate                                  0.323        -0.521   
member_id                                  A000360       B001230   
name                               Lamar Alexander Tammy Baldwin   
congress chamber session roll_call                                 
116      Senate  1       1                     Yes            No   
                         2                     Yes            No   
                         3                     Yes            No   
                         4                     Yes            No   
                         5                      No           Yes   
                         6                      No           Yes   
                         7              Not Voting            No   
                         8                      No           Yes   
                         9                     Yes            No   
                         10                    Yes           Yes   
                         11                    Yes            No   
                         12                    Yes            No   
                         13             Not Voting            No   
                         14                    Yes            No   
                         15                    Yes            No   
                         16                    Yes            No   
                         17                    Yes           Yes   
                         18                    Yes           Yes   
                         19                    Yes           Yes   
                         20                    Yes           Yes   
                         21                    Yes           Yes   
                         22                    Yes           Yes   
                         23                    Yes            No   
                         24                    Yes            No   
                         25                    Yes           Yes   
                         26                    Yes           Yes   
                         27                    Yes            No   
                         28                    Yes            No   
                         29                    Yes            No   
                         30                    Yes           Yes   
...                                            ...           ...   
                         87                    Yes            No   
                         88                    Yes           Yes   
                         89                    Yes           Yes   
                         90                    Yes            No   
                         91                    Yes           Yes   
                         92                    Yes           Yes   
                         93                    Yes            No   
                         94                     No           Yes   
                         95                    Yes            No   
                         96                    Yes           Yes   
                         97                    Yes           Yes   
                         98                    Yes           Yes   
                         99                    Yes            No   
                         100                   Yes           Yes   
                         101                   Yes           Yes   
                         102                   Yes           Yes   
                         103                   Yes            No   
                         104                   Yes            No   
                         105                   Yes            No   
                         106                   Yes            No   
                         107                   Yes            No   
                         108                   Yes      